In [1]:
from datasets import load_dataset

ds = load_dataset("cfilt/iitb-english-hindi")
print(ds)
print(type(ds)) 

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1659083
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 520
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2507
    })
})
<class 'datasets.dataset_dict.DatasetDict'>


In [2]:
ds.keys()

dict_keys(['train', 'validation', 'test'])

In [3]:
ds['train']['translation']

Column([{'en': 'Give your application an accessibility workout', 'hi': 'अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें'}, {'en': 'Accerciser Accessibility Explorer', 'hi': 'एक्सेर्साइसर पहुंचनीयता अन्वेषक'}, {'en': 'The default plugin layout for the bottom panel', 'hi': 'निचले पटल के लिए डिफोल्ट प्लग-इन खाका'}, {'en': 'The default plugin layout for the top panel', 'hi': 'ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका'}, {'en': 'A list of plugins that are disabled by default', 'hi': 'उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है'}, ...])

In [4]:
# Reading data from a parquet file 
from pathlib import Path
data_path: Path = Path("../data/raw/train.parquet")
ds_parquet = load_dataset("parquet", data_files=str(data_path))

In [5]:
ds_parquet.keys()

dict_keys(['train'])

In [6]:
ds_parquet['train']['en']

Column(['Give your application an accessibility workout', 'Accerciser Accessibility Explorer', 'The default plugin layout for the bottom panel', 'The default plugin layout for the top panel', 'A list of plugins that are disabled by default', ...])

In [7]:
import pandas as pd
from pathlib import Path 
from dataclasses import dataclass 
import os 
from typing import Dict, Optional 

# Creating a dataclass to hold the dataset configuration
@dataclass
class DatasetConfig:
    """Configuration class to hold dataset paths and parameters."""
    base_dir:Path = Path("../data")
    raw_dir:Path = base_dir / "raw"
    processed_dir:Path = base_dir / "processed"
    train_file:Path = raw_dir / "train.parquet"
    test_file:Path = raw_dir / "test.parquet"
    validation_file:Path = raw_dir / "validation.parquet"

class IIB_DataLoader:
    """ 
    DataLoader for IITB hindi-english translation dataset 
    -> Responsible for reading parquet files and performing basic integrity checks
    """
    def __init__(self, config:DatasetConfig): 
        self.config = config
        self._validatate_paths() 

    def _validatate_paths(self) -> None: 
        "Internal method to check directories exist or not" 
        if not self.config.raw_dir.exists(): 
            raise FileNotFoundError(f"Raw data directory not found at {self.config.raw_dir}")
    
    def _read_parquet(self, filepath:Path)-> pd.DataFrame: 
        """This reads the parquet files safely"""
      
        if not filepath.exists(): 
            raise FileNotFoundError(f"File not found at {filepath}")
        
        try: 
            df:pd.DataFrame = pd.read_parquet(filepath)
            return df 
        
        except Exception as e:
            raise RuntimeError(f"Error reading parquet file at {filepath}: {e}")
    
    def load_raw_dataset(self)->Dict[str, pd.DataFrame]: 
        """Loads all the splits 'train', 'test' and 'validation' into a dataframe """
        return {
            "train" : self._read_parquet(self.config.train_file),
            "test"  : self._read_parquet(self.config.test_file),
            #"validation" : self._read_parquet(self.config.validation_file) # do not give validation just yet
        }

config:DatasetConfig = DatasetConfig()
data_loader:IIB_DataLoader = IIB_DataLoader(config)
raw_data:Dict[str, pd.DataFrame] = data_loader.load_raw_dataset()

print('sample training data') 
print(raw_data['train'].head())

sample training data
                                               en  \
0  Give your application an accessibility workout   
1               Accerciser Accessibility Explorer   
2  The default plugin layout for the bottom panel   
3     The default plugin layout for the top panel   
4  A list of plugins that are disabled by default   

                                                  hi  
0    अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें  
1                    एक्सेर्साइसर पहुंचनीयता अन्वेषक  
2              निचले पटल के लिए डिफोल्ट प्लग-इन खाका  
3               ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका  
4  उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से नि...  


In [8]:
raw_data['train']['en']

0             Give your application an accessibility workout
1                          Accerciser Accessibility Explorer
2             The default plugin layout for the bottom panel
3                The default plugin layout for the top panel
4             A list of plugins that are disabled by default
                                 ...                        
1659078    The Prime Minister, Shri Narendra Modi has con...
1659079    In a tweet, the Prime Minister said, congratul...
1659080    I also congratulate all those who took oath as...
1659081    The NDA family will work together for the prog...
1659082    I assure all possible support from the Centre ...
Name: en, Length: 1659083, dtype: str

In [9]:
raw_data['train']['hi']

0            अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें
1                            एक्सेर्साइसर पहुंचनीयता अन्वेषक
2                      निचले पटल के लिए डिफोल्ट प्लग-इन खाका
3                       ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका
4          उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से नि...
                                 ...                        
1659078    प्रधानमंत्री श्री नरेन्द्र मोदी ने बिहार के मु...
1659079    एक ट्वीट में प्रधानमंत्री ने कहा, बिहार के मुख...
1659080    मैं उन सभी को भी बधाई देता हूं, जिन्होंने बिहा...
1659081    एनडीए परिवार बिहार की प्रगति के लिए साथ मिलकर ...
1659082    बिहार के कल्याण के लिए केंद्र की ओर से हरसंभव ...
Name: hi, Length: 1659083, dtype: str